In [25]:
import pandas as pd
ratings1= pd.read_csv('data/ratings1.csv', sep=',')
ratings2=pd.read_csv('data/ratings2.csv', sep=',')
# клеим  ratings1 и ratings2 по строкам, так как они имеют одинаковую структуру столбцов. Для этого передадим их списком в функцию concat().
# Помним, что параметр axis по умолчанию равен 0, объединение происходит по строкам, поэтому не трогаем его
ratings=pd.concat([ratings1, ratings2], ignore_index=True)
# читаем movies
movies = pd.read_csv('data/movies.csv', sep= ',')
# читаем dates
dates=pd.read_csv('data/dates.csv', sep=',')
# Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame *drop_duplicates()*, который удаляет повторяющиеся строки в таблице.
# Не забываем обновить индексы после удаления дублей, выставив параметр *ignore_index в методе drop_duplicates()* на значение True:
# Чтобы создать новые индексы, нужно выставить параметр ignore_index на True:
ratings=ratings.drop_duplicates(ignore_index=True)
# добавим к нашей таблице с оценками даты их выставления.
# Для этого конкатенируем таблицы ratings и dates по столбцам:
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(10))

,userId,movieId,rating,date
100826,610,162350,3.5,2017-05-03 22:19:31
100827,610,163937,3.5,2017-05-03 21:59:49
100828,610,163981,3.5,2017-05-03 22:22:35
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


У таблиц ratings и movies есть общий столбец movieId, который каждому фильму из таблицы movies ставит в соответствие поставленные ему оценки из таблицы ratings. Мы хотим объединить их в единую структуру согласно этому соответствию. Объединения такого рода часто называют объединением по ключевому столбцу.

→ Однако прежде чем мы перейдём к дальнейшей работе с нашими таблицами о фильмах, мы должны рассмотреть основные типы объединения таблиц.

                      ТИПЫ ОБЪЕДИНЕНИЙ

Типы объединений в Pandas тесно связаны с операцией join из SQL, которую мы будем рассматривать в курсе в дальнейшем.

Они представлены на схеме ниже в виде кругов Эйлера. 
                    ![f](https://lms.skillfactory.ru/assets/courseware/v1/bb88a7e1138505f1c3b4ce9282a14742/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-u1-md12_7_1.png)

Прежде чем мы перейдём к дальнейшей работе с таблицами о фильмах, рассмотрим два основных типа объединения таблиц:

                                 inner (внутреннее)

При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах.<br/>
  *Аналогия в теории множеств Пересечение (intersection) множеств А и В.*

Строки, для которых совпадение не было найдено, удаляются.

                                outer (внешнее)

Данный тип делится на три подтипа:

 - full — используется как outer по умолчанию, объединяет все варианты в обеих таблицах.<br/>
*Аналогия в теории множеств Объединение (union) множеств А и В*.
 - left — для всех записей из «левой» таблицы (например, ratings) ведётся поиск соответствий в «правой» (например, movies). В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.<br/>
*Аналогия в теории множеств Вычитание (difference) множества B из результата объединения (union) множеств А и В.*

- right — аналогично предыдущему, но остаются значения только из «правой» таблицы. <br/>
*Аналогия в теории множеств Вычитание (difference) множества А из результата объединения (union) множеств А и В.*
____________________________
Во всех трёх случаях, если совпадений между таблицами не найдено, на этом месте ставится пропуск (NaN).

                                          МЕТОД ОБЪЕДИНЕНИЯ JOIN

Для объединения двух таблиц по индексам используется *метод DataFrame join()*. Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это movieId).
__________________________________________________________________
                                 Основные параметры метода join()

- *other* — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».
- *how* — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
- *on* — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».
- *lsuffix и rsuffix* — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.<br/>

Ниже представлена общая схема работы метода join() в зависимости от типа объединения:

![j](https://lms.skillfactory.ru/assets/courseware/v1/e16f8a1193113a12294003598768bbc0/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst3-u1-md12_7_2.png)

Если использовать метод join() «в лоб» (без указания ключевого столбца), то объединение произойдёт, как и задумано — по индексам двух таблиц согласно установленному типу объединения.
_________________
Проверим это, объединив таблицы типом left. Так как в наших таблицах есть одноимённые столбцы, установим один из суффиксов, чтобы избежать ошибки:

In [26]:
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='left'
)
display(joined_false)

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2.0,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3.0,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5.0,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
100831,610,166534,4.0,2017-05-03 21:53:22,NaN,NaN,NaN
100832,610,168248,5.0,2017-05-03 22:21:31,NaN,NaN,NaN
100833,610,168250,5.0,2017-05-08 19:50:47,NaN,NaN,NaN
100834,610,168252,5.0,2017-05-03 21:19:12,NaN,NaN,NaN
